In [1]:
import pandas as pd
#import numpy as np
#import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [2]:
orders = pd.read_csv('NewOrder.csv')
products=pd.read_csv('products.csv')
orders.head()

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [3]:
ordersAisle=pd.merge(orders,products,on=['product_id','product_id'])
ordersAisle.head()

,order_id,product_id,product_name,aisle_id,department_id
0,2,33120,Organic Egg Whites,86,16
1,26,33120,Organic Egg Whites,86,16
2,120,33120,Organic Egg Whites,86,16
3,327,33120,Organic Egg Whites,86,16
4,390,33120,Organic Egg Whites,86,16


In [4]:
#OnlyordersAisle=ordersAisle.drop(['product_id','product_name','department_id'],axis=1)
OnlyordersDepartemnt=ordersAisle.drop(['product_id','product_name','aisle_id'],axis=1)

In [5]:
#products = orders.set_index('order_id')['product_id'].rename('item_id')
#aisles= OnlyordersAisle.set_index('order_id')['aisle_id'].rename('item_id')
departments = OnlyordersDepartemnt.set_index('order_id')['department_id'].rename('item_id')

## Helper functions to the main association rules function

In [6]:
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]

## Association rules function

In [43]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

## Association Rules Mining

In [ ]:
%%time
rulesProuduct = association_rules(products, 0.002)  
rules_final_product=rulesProuduct[['item_A','item_B','lift']]
rules_final_product = rules_final_product.sort_values('lift', ascending=False)
display(rules_final_product)
rules_final_product.to_csv("Association Products Data.csv",index=False)

Starting order_item:               33819106


In [ ]:
%%time
rulesAisles = association_rules(aisles, 0.001)  
rules_final_Aisles = rulesAisles.sort_values('lift', ascending=False)
display(rules_final_Aisles)

In [45]:
%%time
rulesDepartments = association_rules(departments, 0.002)  
rules_final_Departments = rulesDepartments.sort_values('lift', ascending=False)
display(rules_final_Departments)

Starting order_item:               33819106
Items with support >= 0.002:              21
Remaining order_item:              33819106
Remaining orders with 2+ items:     3182490
Remaining order_item:              33655513
Item pairs:                               2
Item pairs with support >= 0.002:          0



,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift


Wall time: 1min 27s


In [22]:
9888378/33819106

0.2923902837644496

In [42]:
departments

order_id
2          16
26         16
120        16
327        16
390        16
537        16
582        16
608        16
623        16
689        16
726        16
771        16
800        16
901        16
1005       16
1097       16
1193       16
1300       16
1887       16
2082       16
2357       16
2551       16
2652       16
2846       16
2929       16
3031       16
3302       16
3304       16
3351       16
3365       16
           ..
3261350     1
3133953     5
3162134     9
3176096    21
1448522    21
2122080    21
3188686    13
111036     13
617924     13
3193791    20
3206343     7
777360      7
3228642    11
3229668     5
3235061    11
3247832    11
3265099    13
3361945    13
3267201    15
3393151    21
3400803     1
774603     19
1351798     7
1832341    17
2169250    16
2269288    21
2507248    21
2271346    17
3004244    21
3376312    21
Name: item_id, Length: 33819106, dtype: int64